In [1]:
%cd ..

import builtins
builtins.uclresearch_topic = 'GIVENCHY' # 226984 entires
# builtins.uclresearch_topic = 'HAWKING' # 4828104 entries
# builtins.uclresearch_topic = 'NYC'
# builtins.uclresearch_topic = 'FLORIDA'
from configuration import config

import smtplib
import base64
from email.mime.text import MIMEText
from email.header import Header
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage
from email.utils import COMMASPACE
import numpy as np
import pandas as pd
import pickle
import xgboost as xgb
import os
import os.path
import math
import sys
import csv
import matplotlib.pyplot as plt
from sklearn.externals import joblib
import scipy.integrate as spi
import pylab as pl
import networkx as nx
from tqdm import tqdm
import multiprocessing
from multiprocessing import Pool
from IPython.display import display

/home/ec2-user/May
('Reading config file from location: '
 '/home/ec2-user/May/configuration/env.properties')
{'calculate': {'analysis': True,
               'network': True,
               'uniquetweets': True,
               'uniqueusers': True},
 'data': {'dates': ['2018-03-11', '2018-03-12', '2018-03-13'],
          'eventname': "Givenchy's Death",
          'phrases': ['givenchy%20death', 'givenchy%20passed%20away'],
          'starttime': 'Mar 12 08:20:00 -0500 2018'},
 'path': {'crawl': {'followers': '/home/ec2-user/May/followers',
                    'friends': '/home/ec2-user/May/friends'},
          'cwd': '/home/ec2-user/May/data/givenchy',
          'ml': '/home/ec2-user/May/data/givenchy/pickle',
          'networkx': {'all': '/home/ec2-user/May/data/givenchy/pickle/networkx_all.dat',
                       'friends': '/home/ec2-user/May/data/givenchy/pickle/networkx_friends.dat',
                       'potential': '/home/ec2-user/May/data/givenchy/pickle/networkx_potenti

/home/ec2-user/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ec2-user/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
n_bytes = 2**31
max_bytes = 2**31 - 1

path = "/home/ec2-user/May/data/givenchy/pickle_old"  

def write_to_pickle(file_path, data):

    bytes_out = pickle.dumps(data)
    with open(file_path, 'wb') as f_out:
        for idx in range(0, n_bytes, max_bytes):
            f_out.write(bytes_out[idx:idx+max_bytes])

def load_pickle(file_path):
    bytes_in = bytearray(0)
    input_size = os.path.getsize(file_path)
    with open(file_path, 'rb') as f_in:
        for _ in range(0, input_size, max_bytes):
            bytes_in += f_in.read(max_bytes)
    return pickle.loads(bytes_in)

def loadModel(self):
    read_file = "data/" + self.model_name + ".dat"
    self.model = joblib.load(read_file)
    return self.model


# Load data

In [3]:
model = joblib.load(path+"/xgb_model_GIVENCHY (2).dat")
unique_users = load_pickle(path+"/users.dat")
network = load_pickle(path+"/network.dat")
users=load_pickle(path+'/Givenchy_features.pkl')

In [4]:
unique_users.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5011 entries, 0 to 5010
Data columns (total 20 columns):
user                                5011 non-null object
user_id                             5011 non-null int64
time_lapsed                         5011 non-null float64
followers_count                     5011 non-null int64
friends_count                       5011 non-null int64
user_created_days                   5011 non-null int64
user_statuses_count                 5011 non-null int64
user_listed_count                   5011 non-null int64
user_favourites_count               5011 non-null int64
normalized_user_statuses_count      5011 non-null float64
normalized_user_followers_count     5011 non-null float64
normalized_user_favourites_count    5011 non-null float64
normalized_user_listed_count        5011 non-null float64
normalized_user_friends_count       5011 non-null float64
mention_and_reply                   5011 non-null object
source_candidates                   501

In [5]:
######## infec_candidates ########
temp=[]
infec = [temp for i in range(len(unique_users))]
unique_users["infec_candidates"]= None
record_num = int(network.describe().iloc[0,0])

for i in range(record_num):
    source_candidates = unique_users.loc[i,'source_candidates']
    #users.loc[i,"infec_candidates"] = list()
    for j in range(len(source_candidates)):
        candidates = source_candidates[j]
        if infec[candidates] == []:
            infec[candidates]=[i]
        else :
            infec[candidates]+=[i]

unique_users.loc[:,"infec_candidates"]=infec

unique_users.head()

,user,user_id,time_lapsed,followers_count,friends_count,user_created_days,user_statuses_count,user_listed_count,user_favourites_count,normalized_user_statuses_count,...,normalized_user_favourites_count,normalized_user_listed_count,normalized_user_friends_count,mention_and_reply,source_candidates,source_index,seed_index,generation,time_since_seed,infec_candidates
0,davelackie,100766356,0.00,143119,4643,2993,181393,555,11938,60.605747,...,3.988640,0.185433,1.551286,[],"[3, 5, 6, 7, 10, 11, 14, 15, 19, 20, 24, 74, 8...",None,0,0,0,"[1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1..."
1,alexanderskhan,3186545203,0.15,753,3428,1042,12853,8,34186,12.334933,...,32.808061,0.007678,3.289827,[0],[0],0,0,1,0.15,"[14, 4334]"
2,consiglierela,4134992843,0.23,2161,396,856,46934,36,78797,54.829439,...,92.052570,0.042056,0.462617,[0],"[0, 12, 21, 32, 48, 748, 819, 972, 2358, 2778,...",0,0,1,0.23,"[12, 32, 818, 1635, 2358, 4699]"
3,ImpactPrincess,1955472014,0.63,242,642,1613,23179,55,28995,14.370118,...,17.975821,0.034098,0.398016,[0],"[0, 115, 577]",0,0,1,0.63,"[0, 3587]"
4,Damek0Masca,406301175,1.13,116,239,2318,2407,12,1058,1.038395,...,0.456428,0.005177,0.103106,[],[],None,4,0,0,[]


# reprocess_data

In [6]:
def reprocess_data(time,origins,infectors,users,unique_users):
        i= int(users[users.user_id.isin([origins])].index.values)
        j= int(users[users.user_id.isin([infectors])].index.values)
        source_candidates = unique_users.loc[j,'source_candidates']
        sources = [x for x in source_candidates if users.loc[x,'TwM_time']<=time]
        sources_time=users[users.index.isin(sources)].sort_values(by=['TwM_time'])
        sources_dataframe = unique_users.loc[sources]
        timeList =  [users.loc[x].TwM_tCurrent - users.loc[x].TwM_time for x in sources]
        degreeList = list(unique_users.loc[k, 'followers_count'] + unique_users.loc[k, 'friends_count']  for k in sources)
    
            
        m=sources_time.index.values[0]
        n=sources_time.index.values[-1]
        
        users.loc[j,'UsM_deltaDays0'] = users.loc[m,'UsM_deltaDays']
        users.loc[j,'UsM_statusesCount0'] = users.loc[m,'UsM_statusesCount']
        users.loc[j,'UsM_followersCount0'] = users.loc[m,'UsM_followersCount']
        users.loc[j,'UsM_favouritesCount0'] = users.loc[m,'UsM_favouritesCount']
        users.loc[j,'UsM_friendsCount0']=users.loc[m,'UsM_friendsCount']
        users.loc[j,'UsM_listedCount0']=users.loc[m,'UsM_listedCount']
        users.loc[j,'UsM_normalizedUserStatusesCount0']=users.loc[m,'UsM_normalizedUserStatusesCount']
        users.loc[j,'UsM_normalizedUserFollowersCount0']=users.loc[m,'UsM_normalizedUserFollowersCount']
        users.loc[j,'UsM_normalizedUserFavouritesCount0']=users.loc[m,'UsM_normalizedUserFavouritesCount']
        users.loc[j,'UsM_normalizedUserListedCount0']=users.loc[m,'UsM_normalizedUserListedCount']
        users.loc[j,'UsM_normalizedUserFriendsCount0']=users.loc[m,'UsM_normalizedUserFriendsCount']
        
        users.loc[j,'UsM_deltaDays-1'] = users.loc[n,'UsM_deltaDays']
        users.loc[j,'UsM_statusesCount-1'] = users.loc[n,'UsM_statusesCount']
        users.loc[j,'UsM_followersCount-1']=users.loc[n,'UsM_followersCount']
        users.loc[j,'UsM_favouritesCount-1']=users.loc[n,'UsM_favouritesCount']
        users.loc[j,'UsM_friendsCount-1']=users.loc[n,'UsM_friendsCount']
        users.loc[j,'UsM_listedCount-1']=users.loc[n,'UsM_listedCount']
        users.loc[j,'UsM_normalizedUserStatusesCount-1']=users.loc[n,'UsM_normalizedUserStatusesCount']
        users.loc[j,'UsM_normalizedUserFollowersCount-1']=users.loc[n,'UsM_normalizedUserFollowersCount']
        users.loc[j,'UsM_normalizedUserFavouritesCount-1']=users.loc[n,'UsM_normalizedUserFavouritesCount']
        users.loc[j,'UsM_normalizedUserListedCount-1']=users.loc[n,'UsM_normalizedUserListedCount']
        users.loc[j,'UsM_normalizedUserFriendsCount-1']=users.loc[n,'UsM_normalizedUserFriendsCount']
        
       
        users.loc[j,'TwM_t0'] = users.loc[m].TwM_tCurrent - users.loc[m,'TwM_time']
        users.loc[j,'TwM_tSeed0'] = users.loc[m,'TwM_tSeed0']
        users.loc[j,'TwM_t-1'] = users.loc[m].TwM_tCurrent - users.loc[n,'TwM_time']
        users.loc[j,'TwM_tSeed-1'] = users.loc[n,'TwM_tSeed-1']
        users.loc[j,'TwM_tCurrent'] =  np.ceil(time/30)*30
        
        
        users.loc[j,'Nw_degree0'] = users.loc[m,'Nw_degree']
        users.loc[j,'Nw_degree-1'] = users.loc[n,'Nw_degree']
        users.loc[j,'Nw_inDegree0'] = users.loc[m,'UsM_friendsCount']
        users.loc[j,'Nw_inDegree-1'] = users.loc[n,'UsM_friendsCount']
        users.loc[j,'Nw_outDegree0'] = users.loc[m,'UsM_followersCount']
        users.loc[j,'Nw_outDegree-1'] = users.loc[n,'UsM_followersCount']
        users.loc[j,'Nw_degreeSeed0'] = users.loc[m,'Nw_degreeSeed0']
        users.loc[j,'Nw_inDegreeSeed0'] = users.loc[m,'Nw_inDegreeSeed0']
        users.loc[j,'Nw_outDegreeSeed0'] = users.loc[m,'Nw_outDegreeSeed0']
        users.loc[j,'Nw_degreeSeed-1'] = users.loc[n,'Nw_degreeSeed-1']
        users.loc[j,'Nw_inDegreeSeed-1'] = users.loc[n,'Nw_inDegreeSeed-1']
        users.loc[j,'Nw_outDegreeSeed-1'] = users.loc[n,'Nw_outDegreeSeed-1']
        
        
        
        users.loc[j,'SNw_generation0'] = users.loc[m,'SNw_generation']
        users.loc[j,'SNw_generation-1'] = users.loc[n,'SNw_generation']
        users.loc[j,'SNw_timeSinceSeed0'] = users.loc[m,'SNw_timeSinceSeed0']
        users.loc[j,'SNw_timeSinceSeed-1'] = users.loc[n,'SNw_timeSinceSeed-1']
        users.loc[j,'SNw_totalNodesInfected'] = infect_totalNodeInfected(users)
        users.loc[j,'SNw_nodeInfectedCentrality'] = len(sources)/users.loc[j,'SNw_totalNodesInfected']
        users.loc[j,'SNw_totalInDegree'] = infect_totalFriends(users)
        users.loc[j,'SNw_totalOutDegree'] = infect_totalFollowers(users)
        users.loc[j,'SNw_inDegreeCentrality'] = in_degree[j]/users.loc[j,'SNw_totalInDegree']
        users.loc[j,'SNw_inDegreeCentrality0'] = users.loc[m,'SNw_inDegreeCentrality']
        users.loc[j,'SNw_inDegreeCentrality-1'] = users.loc[n,'SNw_inDegreeCentrality']
        users.loc[j,'SNw_outDegreeCentrality'] = out_degree[j]/users.loc[j,'SNw_totalOutDegree']
        users.loc[j,'SNw_outDegreeCentrality0'] = users.loc[m,'SNw_outDegreeCentrality']
        users.loc[j,'SNw_outDegreeCentrality-1'] = users.loc[n,'SNw_outDegreeCentrality']
        users.loc[j,'SNw_inDegreeCentralitySeed0'] = users.loc[m,'SNw_inDegreeCentralitySeed0']
        users.loc[j,'SNw_outDegreeCentralitySeed0'] = users.loc[m,'SNw_outDegreeCentralitySeed0']
        users.loc[j,'SNw_inDegreeCentralitySeed-1'] = users.loc[n,'SNw_inDegreeCentralitySeed-1']
        users.loc[j,'SNw_outDegreeCentralitySeed-1'] = users.loc[n,'SNw_outDegreeCentralitySeed-1']


        users.loc[j,'SNw_nFriendsInfected']=len(sources)
        users.loc[j,'SNw_friendsInfectedRatio']=safe_division(len(sources), unique_users.loc[j,'friends_count'])
            
        users.loc[j,'Stat_average_kOut']=round(mean(degreeList), 1)
        users.loc[j,'Stat_average_t']=round(mean(timeList), 1)
        users.loc[j,'Stat_average_deltaDays']=sources_dataframe.user_created_days.mean()
        users.loc[j,'Stat_average_statusesCount']=sources_dataframe.user_statuses_count.mean()
        users.loc[j,'Stat_average_followersCount']=sources_dataframe.followers_count.mean()
        users.loc[j,'Stat_average_favouritesCount']=sources_dataframe.user_favourites_count.mean()
        users.loc[j,'Stat_average_friendsCount']=sources_dataframe.friends_count.mean()
        users.loc[j,'Stat_average_listedCount']=sources_dataframe.user_listed_count.mean()
        users.loc[j,'Stat_average_normalizedUserStatusesCount']=sources_dataframe.normalized_user_statuses_count.mean()
        users.loc[j,'Stat_average_normalizedUserFollowersCount']=sources_dataframe.normalized_user_followers_count.mean()
        users.loc[j,'Stat_average_normalizedUserFavouritesCount']=sources_dataframe.normalized_user_favourites_count.mean()
        users.loc[j,'Stat_average_normalizedUserListedCount']=sources_dataframe.normalized_user_listed_count.mean()
        users.loc[j,'Stat_average_normalizedUserFriendsCount']=sources_dataframe.normalized_user_friends_count.mean()
        if len(degreeList)!=0 :
            users.loc[j,'Stat_max_kOut']=max(degreeList) 
            users.loc[j,'Stat_min_kOut']=min(degreeList) 
        else:
            users.loc[j,'Stat_max_kOut']=''
            users.loc[j,'Stat_min_kOut']=''
#         temp = users[users.index.isin(['j'])]
        return users

In [7]:
# intervals = [60, 30, 15, 7]
intervals = [30]

# Helper function for mean value calculation
def mean(numbers):
    return float(sum(numbers)) / max(len(numbers), 1)

def safe_division(x, y):
    if y == 0:
        return 0
    else:
        return x/y
    
# infectedNetworkStats = {}
# for interval in intervals:
#     statsDictionary = {}
#     for timestamp in range(0, 24*60+1, interval):
#         infected_dataframe = unique_users[unique_users.time_lapsed <= timestamp]
#         statsDictionary[timestamp] = {}
#         statsDictionary[timestamp]['totalNodeInfected'] = infected_dataframe.shape[0]
#         statsDictionary[timestamp]['totalFollowers'] = sum(infected_dataframe.followers_count)
#         statsDictionary[timestamp]['totalFriends'] = sum(infected_dataframe.friends_count)
#     infectedNetworkStats[interval] = statsDictionary

def infect_totalNodeInfected(users):
    infected_dataframe = users[users.TwM_time>=0]
    states = infected_dataframe.shape[0]
    return states

def infect_totalFollowers(users):
    infected_dataframe = users[users.TwM_time>=0]
    states = sum(infected_dataframe.UsM_followersCount)
    return states

def infect_totalFriends(users):
    infected_dataframe = users[users.TwM_time>=0]
    states = sum(infected_dataframe.UsM_friendsCount)
    return states

in_degree = list(unique_users.friends_count)
out_degree = list(unique_users.followers_count)
degree = [sum(x) for x in zip(in_degree, out_degree)]

In [8]:
network_simulation = pd.DataFrame(columns=["t_current","user_id","source_candidates","infec_candidates",
                                           "followers_count","friends_count","source_id","seed_user_id",
                                           "SIR","generations","time_lapsed","time_since_seed"])
# network_simulation["user"] =unique_users["user"]
network_simulation["user_id"] = unique_users["user_id"]
network_simulation["source_candidates"] = unique_users["source_candidates"]
network_simulation["infec_candidates"] = unique_users["infec_candidates"]
network_simulation["followers_count"] = unique_users["followers_count"]
network_simulation["friends_count"] = unique_users["friends_count"]
#network_simulation.head()

In [9]:
######## All Seeds 99 ########
seeds = network.copy()

record_num_network = network.describe().iloc[0,0]
record_num_network = int(record_num_network)
for i in range(record_num_network):
    record = network.loc[i,"generation"]
    if record != 0:
        seeds.drop([i],axis=0, inplace = True)

In [10]:
######## Effective Seeds 24 ########
effective_seeds= seeds.copy()

record_num_seeds = seeds.describe().iloc[0,0]
record_num_seeds = int(record_num_seeds)
for i in range(record_num_seeds):
    k=0
    user_index = seeds.index[i]
    user_id = seeds.loc[user_index,"user_id"]
    for j in range(record_num_network):
        source_id = network.loc[j,"source_id"]
        if user_id == source_id:
            k=k+1
            #print(k)
    if k == 0:
        effective_seeds.drop([user_index],axis=0, inplace = True)

In [11]:
### seeds ###
def seeds_setting_simulation(seeds,network_simulation):
    for i in range(len(seeds)):
        index = seeds.index.values
        network_simulation.loc[index,'SIR'] = 1
        network_simulation.loc[index,'generations'] = 0
        network_simulation.loc[index,'source_id'] = seeds.loc[index,'user_id']
        network_simulation.loc[index,'time_lapsed'] = seeds.loc[index,'time_lapsed']
        network_simulation.loc[index,'seed_user_id'] = seeds.loc[index,'user_id']
        network_simulation.loc[index,'time_since_seed'] = 0
    return network_simulation

def seeds_setting_features(seeds,users):
    for i in range(len(seeds)):
     
        j = seeds.index.values[i]
#         sources = [x for x in seeds.loc[j,'source_candidates']]

        users.loc[j,'UsM_deltaDays0'] = users.loc[j,'UsM_deltaDays']
        users.loc[j,'UsM_statusesCount0'] = users.loc[j,'UsM_statusesCount']
        users.loc[j,'UsM_followersCount0']=users.loc[j,'UsM_followersCount']
        users.loc[j,'UsM_favouritesCount0']=users.loc[j,'UsM_favouritesCount']
        users.loc[j,'UsM_friendsCount0']=users.loc[j,'UsM_friendsCount']
        users.loc[j,'UsM_listedCount0']=users.loc[j,'UsM_listedCount']
        users.loc[j,'UsM_normalizedUserStatusesCount0']=users.loc[j,'UsM_normalizedUserStatusesCount']
        users.loc[j,'UsM_normalizedUserFollowersCount0']=users.loc[j,'UsM_normalizedUserFollowersCount']
        users.loc[j,'UsM_normalizedUserFavouritesCount0']=users.loc[j,'UsM_normalizedUserFavouritesCount']
        users.loc[j,'UsM_normalizedUserListedCount0']=users.loc[j,'UsM_normalizedUserListedCount']
        users.loc[j,'UsM_normalizedUserFriendsCount0']=users.loc[j,'UsM_normalizedUserFriendsCount']
        
        users.loc[j,'UsM_deltaDays-1'] = users.loc[j,'UsM_deltaDays']
        users.loc[j,'UsM_statusesCount-1'] = users.loc[j,'UsM_statusesCount']
        users.loc[j,'UsM_followersCount-1']=users.loc[j,'UsM_followersCount']
        users.loc[j,'UsM_favouritesCount-1']=users.loc[j,'UsM_favouritesCount']
        users.loc[j,'UsM_friendsCount-1']=users.loc[j,'UsM_friendsCount']
        users.loc[j,'UsM_listedCount-1']=users.loc[j,'UsM_listedCount']
        users.loc[j,'UsM_normalizedUserStatusesCount-1']=users.loc[j,'UsM_normalizedUserStatusesCount']
        users.loc[j,'UsM_normalizedUserFollowersCount-1']=users.loc[j,'UsM_normalizedUserFollowersCount']
        users.loc[j,'UsM_normalizedUserFavouritesCount-1']=users.loc[j,'UsM_normalizedUserFavouritesCount']
        users.loc[j,'UsM_normalizedUserListedCount-1']=users.loc[j,'UsM_normalizedUserListedCount']
        users.loc[j,'UsM_normalizedUserFriendsCount-1']=users.loc[j,'UsM_normalizedUserFriendsCount']
        
        users.loc[j,'TwM_t0'] =seeds.loc[j,'time_lapsed']
        users.loc[j,'TwM_t-1'] =seeds.loc[j,'time_lapsed']
        users.loc[j,'TwM_tCurrent'] = np.ceil(seeds.loc[j,'time_lapsed']+0.001/30)*30   
        users.loc[j,'TwM_tSeed0'] = seeds.loc[j,'time_lapsed']                         
        users.loc[j,'TwM_tSeed-1'] = seeds.loc[j,'time_lapsed']
        users.loc[j,'TwM_time'] = seeds.loc[j,'time_lapsed']

        users.loc[j,'Nw_degree0'] = users.loc[j,'Nw_degree']
        users.loc[j,'Nw_degree-1'] = users.loc[j,'Nw_degree']
        users.loc[j,'Nw_inDegree0'] = users.loc[j,'UsM_friendsCount']
        users.loc[j,'Nw_inDegree-1'] = users.loc[j,'UsM_friendsCount']
        users.loc[j,'Nw_outDegree0'] = users.loc[j,'UsM_followersCount']
        users.loc[j,'Nw_outDegree-1'] = users.loc[j,'UsM_followersCount']
        users.loc[j,'Nw_degreeSeed0'] = users.loc[j,'Nw_degree']    
        users.loc[j,'Nw_inDegreeSeed0'] = users.loc[j,'UsM_friendsCount'] 
        users.loc[j,'Nw_outDegreeSeed0'] = users.loc[j,'UsM_followersCount']
        users.loc[j,'Nw_degreeSeed-1'] = users.loc[j,'Nw_degree']
        users.loc[j,'Nw_inDegreeSeed-1'] = users.loc[j,'UsM_friendsCount']
        users.loc[j,'Nw_outDegreeSeed-1'] = users.loc[j,'UsM_followersCount']
        
        users.loc[j,'SNw_generation0'] = 0
        users.loc[j,'SNw_generation-1'] = 0
        users.loc[j,'SNw_generation'] = 0
        users.loc[j,'SNw_totalNodesInfected'] = infect_totalNodeInfected(users)
#         users.loc[j,'SNw_nodeInfectedCentrality'] = len(sources)/users.loc[j,'SNw_totalNodesInfected']
        users.loc[j,'SNw_totalInDegree'] = infect_totalFriends(users)
        users.loc[j,'SNw_totalOutDegree'] = infect_totalFollowers(users)
        users.loc[j,'SNw_inDegreeCentrality'] = in_degree[j]/users.loc[j,'SNw_totalInDegree']
        users.loc[j,'SNw_inDegreeCentrality0'] = users.loc[j,'SNw_inDegreeCentrality']
        users.loc[j,'SNw_inDegreeCentrality-1'] = users.loc[j,'SNw_inDegreeCentrality']
        users.loc[j,'SNw_outDegreeCentrality'] = out_degree[j]/users.loc[j,'SNw_totalOutDegree']
        users.loc[j,'SNw_outDegreeCentrality0'] = users.loc[j,'SNw_outDegreeCentrality']
        users.loc[j,'SNw_outDegreeCentrality-1'] = users.loc[j,'SNw_outDegreeCentrality']
        users.loc[j,'SNw_inDegreeCentralitySeed0'] = users.loc[j,'SNw_inDegreeCentrality']               
        users.loc[j,'SNw_outDegreeCentralitySeed0'] = users.loc[j,'SNw_outDegreeCentrality']                
        users.loc[j,'SNw_inDegreeCentralitySeed-1'] = users.loc[j,'SNw_inDegreeCentrality']                  
        users.loc[j,'SNw_outDegreeCentralitySeed-1'] = users.loc[j,'SNw_outDegreeCentrality']
    return users

In [12]:
seeds_setting_simulation(seeds,network_simulation)
seeds_setting_features(seeds,users)

,label,UsM_deltaDays,UsM_statusesCount,UsM_followersCount,UsM_favouritesCount,UsM_friendsCount,UsM_listedCount,UsM_normalizedUserStatusesCount,UsM_normalizedUserFollowersCount,UsM_normalizedUserFavouritesCount,...,Stat_average_friendsCount,Stat_average_listedCount,Stat_average_normalizedUserStatusesCount,Stat_average_normalizedUserFollowersCount,Stat_average_normalizedUserFavouritesCount,Stat_average_normalizedUserListedCount,Stat_average_normalizedUserFriendsCount,Stat_max_kOut,Stat_min_kOut,user_id
0,NaN,2993,181393,143119,11938,4643,555,60.6057,47.8179,3.98864,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100766356
1,NaN,1042,12853,753,34186,3428,8,12.3349,0.722649,32.8081,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3186545203
2,NaN,856,46934,2161,78797,396,36,54.8294,2.52453,92.0526,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4134992843
3,NaN,1613,23179,242,28995,642,55,14.3701,0.150031,17.9758,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1955472014
4,NaN,2318,2407,116,1058,239,12,1.0384,0.0500431,0.456428,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,406301175
5,NaN,1176,18995,250,13390,493,20,16.1522,0.212585,11.3861,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2938214163
6,NaN,3041,34644,39697,20885,1924,528,11.3923,13.0539,6.86781,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89805327
7,NaN,2421,15651,672,54298,353,1,6.46468,0.277571,22.4279,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,342931336
8,NaN,3272,25944,275,13821,440,21,7.9291,0.0840465,4.22402,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27220084
9,NaN,3008,175675,1911,3810,506,80,58.4026,0.635306,1.26662,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97360017


In [13]:
#users.info()

In [14]:

def simulation(t,unique_users,network_simulation):
    for i in range(len(unique_users)):
        if np.isnan(network_simulation.loc[i,"time_lapsed"]) == False :
            if round(network_simulation.loc[i,"time_lapsed"]) <= t:
                infec_candidates=network_simulation.loc[i,"infec_candidates"]

                for j in range(len(infec_candidates)):
                    infector = network_simulation.loc[i,"infec_candidates"][j]

                    if network_simulation.loc[infector,'SIR']!=1 :
                        user_id1 = network_simulation.loc[i,'user_id']
                        user_id2 = network_simulation.loc[infector,'user_id']
                        processed_user=reprocess_data(t+1,user_id1,user_id2,users,unique_users)

                        valid = processed_user[processed_user.index.isin([infector])]
#                             print(valid['Stat_average_t'])
                        valid = valid.drop(['label','user_id','SNw_generation','TwM_time'],axis=1)

                        valid = valid.astype('float64')

                        columns = list(valid.columns)
                        X = valid[columns]
                        pre_data = xgb.DMatrix(X)
                        infec = model.predict(pre_data)
#                         print(infec)
                        if infec>0.3 :
                            users.loc[infector,'TwM_time'] = t
                            network_simulation.loc[infector,"SIR"] = 1
                            network_simulation.loc[infector,"generations"] = network_simulation.loc[i,"generations"]+1
                            network_simulation.loc[infector,"source_id"] = network_simulation.loc[i,"user_id"] 
                            network_simulation.loc[infector,"seed_user_id"] = network_simulation.loc[i,"seed_user_id"] 
                            network_simulation.loc[infector,"time_lapsed"] = t+1
            #             network_simulation.loc[i,"SIR"]=2 
    return unique_users,network_simulation


In [15]:
##time = 5*60 minutes with resuming, due to releasing memory
for t in range(0,10):
    temp_data=simulation(t,unique_users,network_simulation)
    unique_users = temp_data[0]
    network_simulation = temp_data[1]
    print(t)
    write_to_pickle(path+'/Givenchy_network_simulation.pkl', network_simulation)
    write_to_pickle(path+'/Givenchy_users_features.pkl', users)

for t in range(10,20):
    temp_data=simulation(t,unique_users,network_simulation)
    unique_users = temp_data[0]
    network_simulation = temp_data[1]
    print(t)
    write_to_pickle(path+'/Givenchy_network_simulation.pkl', network_simulation)
    write_to_pickle(path+'/Givenchy_users_features.pkl', users)
    
for t in range(20,30):
    temp_data=simulation(t,unique_users,network_simulation)
    unique_users = temp_data[0]
    network_simulation = temp_data[1]
    print(t)
    write_to_pickle(path+'/Givenchy_network_simulation.pkl', network_simulation)
    write_to_pickle(path+'/Givenchy_users_features.pkl', users)
    
for t in range(30,40):
    temp_data=simulation(t,unique_users,network_simulation)
    unique_users = temp_data[0]
    network_simulation = temp_data[1]
    print(t)
    write_to_pickle(path+'/Givenchy_network_simulation.pkl', network_simulation)
    write_to_pickle(path+'/Givenchy_users_features.pkl', users)

for t in range(40,50):
    temp_data=simulation(t,unique_users,network_simulation)
    unique_users = temp_data[0]
    network_simulation = temp_data[1]
    print(t)
    write_to_pickle(path+'/Givenchy_network_simulation.pkl', network_simulation)
    write_to_pickle(path+'/Givenchy_users_features.pkl', users)
    
for t in range(50,60):
    temp_data=simulation(t,unique_users,network_simulation)
    unique_users = temp_data[0]
    network_simulation = temp_data[1]
    print(t)
    write_to_pickle(path+'/Givenchy_network_simulation.pkl', network_simulation)
    write_to_pickle(path+'/Givenchy_users_features.pkl', users)
    
for t in range(70,80):
    temp_data=simulation(t,unique_users,network_simulation)
    unique_users = temp_data[0]
    network_simulation = temp_data[1]
    print(t)
    write_to_pickle(path+'/Givenchy_network_simulation.pkl', network_simulation)
    write_to_pickle(path+'/Givenchy_users_features.pkl', users)
    
for t in range(90,100):
    temp_data=simulation(t,unique_users,network_simulation)
    unique_users = temp_data[0]
    network_simulation = temp_data[1]
    print(t)
    write_to_pickle(path+'/Givenchy_network_simulation.pkl', network_simulation)
    write_to_pickle(path+'/Givenchy_users_features.pkl', users)
    
for t in range(110,120):
    temp_data=simulation(t,unique_users,network_simulation)
    unique_users = temp_data[0]
    network_simulation = temp_data[1]
    print(t)
    write_to_pickle(path+'/Givenchy_network_simulation.pkl', network_simulation)
    write_to_pickle(path+'/Givenchy_users_features.pkl', users)
    
for t in range(120,130):
    temp_data=simulation(t,unique_users,network_simulation)
    unique_users = temp_data[0]
    network_simulation = temp_data[1]
    print(t)
    write_to_pickle(path+'/Givenchy_network_simulation.pkl', network_simulation)
    write_to_pickle(path+'/Givenchy_users_features.pkl', users)
    
for t in range(130,140):
    temp_data=simulation(t,unique_users,network_simulation)
    unique_users = temp_data[0]
    network_simulation = temp_data[1]
    print(t)
    write_to_pickle(path+'/Givenchy_network_simulation.pkl', network_simulation)
    write_to_pickle(path+'/Givenchy_users_features.pkl', users)
    
for t in range(140,150):
    temp_data=simulation(t,unique_users,network_simulation)
    unique_users = temp_data[0]
    network_simulation = temp_data[1]
    print(t)
    write_to_pickle(path+'/Givenchy_network_simulation.pkl', network_simulation)
    write_to_pickle(path+'/Givenchy_users_features.pkl', users)
    
for t in range(150,160):
    temp_data=simulation(t,unique_users,network_simulation)
    unique_users = temp_data[0]
    network_simulation = temp_data[1]
    print(t)
    write_to_pickle(path+'/Givenchy_network_simulation.pkl', network_simulation)
    write_to_pickle(path+'/Givenchy_users_features.pkl', users)
    
for t in range(160,170):
    temp_data=simulation(t,unique_users,network_simulation)
    unique_users = temp_data[0]
    network_simulation = temp_data[1]
    print(t)
    write_to_pickle(path+'/Givenchy_network_simulation.pkl', network_simulation)
    write_to_pickle(path+'/Givenchy_users_features.pkl', users)
    
for t in range(170,180):
    temp_data=simulation(t,unique_users,network_simulation)
    unique_users = temp_data[0]
    network_simulation = temp_data[1]
    print(t)
    write_to_pickle(path+'/Givenchy_network_simulation.pkl', network_simulation)
    write_to_pickle(path+'/Givenchy_users_features.pkl', users)
    
for t in range(180,190):
    temp_data=simulation(t,unique_users,network_simulation)
    unique_users = temp_data[0]
    network_simulation = temp_data[1]
    print(t)
    write_to_pickle(path+'/Givenchy_network_simulation.pkl', network_simulation)
    write_to_pickle(path+'/Givenchy_users_features.pkl', users)
    
for t in range(190,200):
    temp_data=simulation(t,unique_users,network_simulation)
    unique_users = temp_data[0]
    network_simulation = temp_data[1]
    print(t)
    write_to_pickle(path+'/Givenchy_network_simulation.pkl', network_simulation)
    write_to_pickle(path+'/Givenchy_users_features.pkl', users)
    
for t in range(210,220):
    temp_data=simulation(t,unique_users,network_simulation)
    unique_users = temp_data[0]
    network_simulation = temp_data[1]
    print(t)
    write_to_pickle(path+'/Givenchy_network_simulation.pkl', network_simulation)
    write_to_pickle(path+'/Givenchy_users_features.pkl', users)
    
for t in range(220,230):
    temp_data=simulation(t,unique_users,network_simulation)
    unique_users = temp_data[0]
    network_simulation = temp_data[1]
    print(t)
    write_to_pickle(path+'/Givenchy_network_simulation.pkl', network_simulation)
    write_to_pickle(path+'/Givenchy_users_features.pkl', users)
    
for t in range(230,240):
    temp_data=simulation(t,unique_users,network_simulation)
    unique_users = temp_data[0]
    network_simulation = temp_data[1]
    print(t)
    write_to_pickle(path+'/Givenchy_network_simulation.pkl', network_simulation)
    write_to_pickle(path+'/Givenchy_users_features.pkl', users)
    
for t in range(240,250):
    temp_data=simulation(t,unique_users,network_simulation)
    unique_users = temp_data[0]
    network_simulation = temp_data[1]
    print(t)
    write_to_pickle(path+'/Givenchy_network_simulation.pkl', network_simulation)
    write_to_pickle(path+'/Givenchy_users_features.pkl', users)
    
for t in range(250,260):
    temp_data=simulation(t,unique_users,network_simulation)
    unique_users = temp_data[0]
    network_simulation = temp_data[1]
    print(t)
    write_to_pickle(path+'/Givenchy_network_simulation.pkl', network_simulation)
    write_to_pickle(path+'/Givenchy_users_features.pkl', users)
    
for t in range(260,270):
    temp_data=simulation(t,unique_users,network_simulation)
    unique_users = temp_data[0]
    network_simulation = temp_data[1]
    print(t)
    write_to_pickle(path+'/Givenchy_network_simulation.pkl', network_simulation)
    write_to_pickle(path+'/Givenchy_users_features.pkl', users)
    
for t in range(270,280):
    temp_data=simulation(t,unique_users,network_simulation)
    unique_users = temp_data[0]
    network_simulation = temp_data[1]
    print(t)
    write_to_pickle(path+'/Givenchy_network_simulation.pkl', network_simulation)
    write_to_pickle(path+'/Givenchy_users_features.pkl', users)
    
for t in range(280,290):
    temp_data=simulation(t,unique_users,network_simulation)
    unique_users = temp_data[0]
    network_simulation = temp_data[1]
    print(t)
    write_to_pickle(path+'/Givenchy_network_simulation.pkl', network_simulation)
    write_to_pickle(path+'/Givenchy_users_features.pkl', users)
    
for t in range(290,300):
    temp_data=simulation(t,unique_users,network_simulation)
    unique_users = temp_data[0]
    network_simulation = temp_data[1]
    print(t)
    write_to_pickle(path+'/Givenchy_network_simulation.pkl', network_simulation)
    write_to_pickle(path+'/Givenchy_users_features.pkl', users)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
70
71
72
73
74
75
76
77
78
79
90
91
92
93
94
95
96
97
98
99
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130


KeyboardInterrupt: 

In [ ]:
network_simulation["t_current"]=users["TwM_tCurrent"]
# network_simulation

In [23]:
# network_simulation[network_simulation.generations.isin([0])]
network_simulation[network_simulation.generations.isin([8])]

# network_simulation.info()

,t_current,user_id,source_candidates,infec_candidates,followers_count,friends_count,source_id,seed_user_id,SIR,generations,time_lapsed,time_since_seed


In [24]:
write_to_pickle(path+'/Givenchy_network_simulation.pkl', network_simulation)
write_to_pickle(path+'/Givenchy_users_features.pkl', users)

In [ ]:
# network_simulation